In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import os

## Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame

In [3]:
# open csv form part 1
city_data_df = pd.read_csv(os.path.join("weather_data","WeatherPy_challenge.csv"))
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date
0,0,Qaanaaq,77.48,-69.36,-14.51,58,0,5.61,GL,clear sky,0.00,0.0,2020-02-20 22:46:30
1,1,Hobart,-42.88,147.33,61.00,44,40,13.87,AU,light rain,0.25,0.0,2020-02-20 22:46:30
2,2,Albany,42.60,-73.97,26.60,38,1,16.11,US,clear sky,0.00,0.0,2020-02-20 22:45:03
3,3,Port Alfred,-33.59,26.89,68.00,92,59,6.04,ZA,broken clouds,0.00,0.0,2020-02-20 22:46:31
4,4,Najran,17.49,44.13,73.40,35,53,4.70,SA,broken clouds,0.00,0.0,2020-02-20 22:46:32


## Filter the DataFrame for minimum and maximum temperature preferences and rain/snow preference

In [50]:
# a little google foo from https://gist.github.com/garrettdreyfus/8153571
def ask_user(question):
    check = str(input(question)).lower().strip()
    try:
        if check[0] == 'y':
            return True
        elif check[0] == 'n':
            return False
        else:
            print('Invalid Input')
            return ask_user(question)
    except Exception as error:
        print("Please enter Yes or No or I will keep asking for a valid input.")
        return ask_user(question)

In [32]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_pref = ask_user("Do you want it to be raining? (yes/no) ")
snow_pref = ask_user("Do you want it to be snowing? (yes/no) ")

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
# if true use cities recorded rain
if rain_pref == True:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain"] > 0 )]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain"] <= 0 )]

# if true use cities recorded snow
if snow_pref == True:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow"] > 0 )]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow"] <= 0 )]

preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date
11,11,Rikitea,-23.12,-134.97,78.98,74,43,10.71,PF,light rain,0.50,0.0,2020-02-20 22:46:36
29,29,Ballina,-28.87,153.57,77.00,83,79,6.93,AU,light rain,1.94,0.0,2020-02-20 22:46:45
34,34,Novo Aripuana,-5.12,-60.38,76.10,97,100,1.16,BR,light rain,1.44,0.0,2020-02-20 22:46:47
38,38,Mataura,-46.19,168.86,72.39,63,82,7.65,NZ,light rain,0.31,0.0,2020-02-20 22:46:49
42,42,Katobu,-4.94,122.53,76.50,92,100,1.12,ID,light rain,0.19,0.0,2020-02-20 22:46:51
49,49,Gizo,-8.10,156.84,81.09,78,49,6.04,SB,light rain,1.81,0.0,2020-02-20 22:46:55
55,55,Saint-Philippe,-21.36,55.77,77.00,69,20,3.36,RE,few clouds,0.63,0.0,2020-02-20 22:43:16
57,57,Carnarvon,-24.87,113.63,78.80,74,40,9.17,AU,scattered clouds,0.81,0.0,2020-02-20 22:47:01
61,61,Itarema,-2.92,-39.92,76.08,93,100,3.29,BR,light rain,0.25,0.0,2020-02-20 22:47:03
69,69,Airai,-8.93,125.41,73.26,80,87,1.36,TL,light rain,0.12,0.0,2020-02-20 22:47:08


## Add the cities to a marker layer map with a pop-up marker for each city:

In [33]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [34]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Description,Hotel Name
11,Rikitea,PF,78.98,-23.12,-134.97,light rain,
29,Ballina,AU,77.00,-28.87,153.57,light rain,
34,Novo Aripuana,BR,76.10,-5.12,-60.38,light rain,
38,Mataura,NZ,72.39,-46.19,168.86,light rain,
42,Katobu,ID,76.50,-4.94,122.53,light rain,
49,Gizo,SB,81.09,-8.10,156.84,light rain,
55,Saint-Philippe,RE,77.00,-21.36,55.77,few clouds,
57,Carnarvon,AU,78.80,-24.87,113.63,scattered clouds,
61,Itarema,BR,76.08,-2.92,-39.92,light rain,
69,Airai,TL,73.26,-8.93,125.41,light rain,


In [35]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


## Your new hotel DataFrame should look similar to the challenge image:

In [41]:
# Rename the columns
hotel_df.rename(columns={"Description": "Current Description"}, inplace=True)

#change the order
new_column_order = ['City', 'Country', 'Max Temp', 'Current Description', 'Lat', 'Lng', 'Hotel Name']
hotel_df = hotel_df[new_column_order]

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Rikitea,PF,78.98,light rain,-23.12,-134.97,Pension Maro'i
29,Ballina,AU,77.00,light rain,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron
34,Novo Aripuana,BR,76.10,light rain,-5.12,-60.38,Hotel Ypê
38,Mataura,NZ,72.39,light rain,-46.19,168.86,Ellie's Villa
42,Katobu,ID,76.50,light rain,-4.94,122.53,La Ode Gombe
49,Gizo,SB,81.09,light rain,-8.10,156.84,Imagination Island
55,Saint-Philippe,RE,77.00,few clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
57,Carnarvon,AU,78.80,scattered clouds,-24.87,113.63,Hospitality Carnarvon
61,Itarema,BR,76.08,light rain,-2.92,-39.92,Windvillage
69,Airai,TL,73.26,light rain,-8.93,125.41,Juvinal Martin


## Your marker layer map should look similar to the challenge image:

In [37]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>Max Temp: {Max Temp} °F, {Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [52]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Save the data to CSV

In [44]:
# Create the output file (CSV).
output_data_file = os.path.join("weather_data","WeatherPy_vacation.csv")
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")